In [1]:
import time
import os
import numpy as np
import pandas as pd
from module import in_fun
from matplotlib import pyplot as plt
from module.model import AnnModel
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

%matplotlib widget

In [2]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

print(device)

cpu


In [3]:
net_ble = AnnModel(input_size = 80)
net_wifi = AnnModel(input_size = 80)

net_ble.load_state_dict(torch.load("./model/ble"))
net_wifi.load_state_dict(torch.load("./model/wifi"))

<All keys matched successfully>

In [4]:
data_path = "data/ref/210423/"
file_list = os.listdir(data_path)
file_list_csv = [file for file in file_list if file.endswith(".csv")]
print(file_list_csv)

['00_confirmed.csv', '01_1250.csv', '02_1233.csv', '03_1210.csv', '04_1239.csv', '05_1241.csv', '06_1251_cs.csv', '07_1251_th.csv']


In [5]:
data_list = []
for file_csv in file_list_csv:
    data_list.append(pd.read_csv(
            data_path + file_csv,
            header=None,
        ).to_numpy())

In [6]:
plt.figure()
for idx, data in enumerate(data_list):
    plt.scatter(pd.to_datetime(data[:,0]/1000, unit='s'), np.ones_like(data[:,0])*(idx+1), label = file_list_csv[idx], s = 5)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
plt.figure()
data=data_list[2]
plt.scatter(pd.to_datetime(data[:,0]/1000, unit='s'), data[:,42], s = 5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# for idx, data in enumerate(data_list):
    print(f"{file_list_csv[idx][:-4]} start time : {datetime.fromtimestamp(data[0,0]/1000)}")

In [8]:
start_time = datetime(2021, 4, 23, hour=6, minute=20, second=0)
end_time = datetime(2021, 4, 23, hour=9, minute=20, second=0)
start_timestamp = time.mktime(start_time.timetuple())
end_timestamp = time.mktime(end_time.timetuple())
print(start_time)
print(start_timestamp)
print(end_time)
print(end_timestamp)
for idx in np.arange(len(data_list)):
    print(np.shape(data_list[idx]))
    data_list[idx] = data_list[idx][data_list[idx][:,0] >= start_timestamp * 1000]
    data_list[idx] = data_list[idx][data_list[idx][:,0] <= end_timestamp * 1000]
    
    print(np.shape(data_list[idx]))

2021-04-23 06:20:00
1619158800.0
2021-04-23 09:20:00
1619169600.0
(377, 101)
(360, 101)
(393, 101)
(360, 101)
(387, 101)
(360, 101)
(398, 101)
(360, 101)
(399, 101)
(360, 101)
(398, 101)
(360, 101)
(382, 101)
(360, 101)
(380, 101)
(360, 101)


In [9]:
plt.figure()
for idx, data in enumerate(data_list):
    plt.scatter(pd.to_datetime(data[:,0]/1000, unit='s'), np.ones_like(data[:,0])*(idx+1), label = file_list_csv[idx], s=5)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
const_val = -200
start_time = time.time()
data_list
for idx in range(np.shape(data_list[0])[0]):
    confirmed_time = data_list[0][idx, 0]
    confirmed_ble = data_list[0][idx, 1:41] 
    confirmed_wifi = data_list[0][idx, 41:81]
    # confirmed part
    ## ble part
    if confirmed_ble[1] == 0:
        print("no confirmed ble")
        confirmed_ble_index = None
    else:
        confirmed_ble_index = True
        
    ## wifi part
    if confirmed_wifi[1] == 0:
        print("no confirmed wifi")
        confirmed_wifi_index = None
    else:
        confirmed_wifi_index = True
    # contact part
    
    ## ble part
    result_ble = []
    if confirmed_ble_index is not None:
        for contact in data_list[1:]:
            contact_ble = contact[idx, 1:41] 
            if contact_ble[1] == 0:
                result_ble.append("no contact ble")
            else:
                inp_ble = in_fun.test_matcher(confirmed_ble, contact_ble)
                logit_ble = net_ble(torch.from_numpy(inp_ble).float())
                result = torch.sigmoid(logit_ble) >= 0.5
                result_ble.append(result)
                
    ## wifi part
    result_wifi = []
    if confirmed_wifi_index is not None:
        for contact in data_list[1:]:
            contact_wifi = contact[idx, 41:81]
            if contact_wifi[1] == 0:
                result_wifi.append("no contact wifi")
            else:
                inp_diff = in_fun.test_matcher(confirmed_wifi, contact_wifi)


In [12]:
const_val = -200
start_time = time.time()
for idx in range(np.shape(data_list[0])[0]):
    confirmed_time = data_list[0][idx, 0]
    confirmed_ble = data_list[0][idx, 1:41] 
    confirmed_wifi = data_list[0][idx, 41:81]
    # confirmed part
    ## ble part
    if confirmed_ble[1] == 0:
        print("no confirmed ble")
        confirmed_ble_index = None
    else:
        confirmed_ble_index = True
        
    ## wifi part
    if confirmed_wifi[1] == 0:
        print("no confirmed wifi")
        confirmed_wifi_index = None
    else:
        confirmed_wifi_index = True
    # contact part
    
    ## ble part
    result_ble = []
    if confirmed_ble_index is not None:
        for contact in data_list[1:]:
            contact_ble = contact[idx, 1:41] 
            if contact_ble[1] == 0:
                result_ble.append("no contact ble")
            else:
                inp_ble = in_fun.test_matcher(confirmed_ble, contact_ble)
                hypothesis_ble = net_ble(torch.from_numpy(inp_ble).float())
                result = torch.sigmoid(hypothesis_ble) >= 0.5
                result_ble.append(result)
                
    ## wifi part
    result_wifi = []
    if confirmed_wifi_index is not None:
        for contact in data_list[1:]:
            contact_wifi = contact[idx, 41:81]
            if contact_wifi[1] == 0:
                result_wifi.append("no contact wifi")
            else:
                inp_wifi = in_fun.test_matcher(confirmed_wifi, contact_wifi)
                hypothsis_wifi = net_wifi(torch.from_numpy(inp_wifi).float())
                result = torch.sigmoid(hypothsis_wifi) >= 0.5
                result_wifi.append(result)
        
    print(f"{idx} 시각 : {pd.to_datetime(confirmed_time/1000, unit='s')}")
    if len(result_ble) > 0:
        for idx, result in enumerate(result_ble):
            if result == True:
                print(f"{file_list_csv[idx+1]} contacted by ble")
    if len(result_wifi) > 0:
        for idx, result in enumerate(result_wifi):
            if result == True:
#                 print(f"{file_list_csv[idx+1]} contacted by wifi")
                pass
    
print(f"총 소요시간 : {time.time()-start_time}초")

0 시각 : 2021-04-23 06:20:00.002000128
01_1250.csv contacted by ble
07_1251_th.csv contacted by ble
1 시각 : 2021-04-23 06:20:30.002000128
01_1250.csv contacted by ble
07_1251_th.csv contacted by ble
2 시각 : 2021-04-23 06:21:00.002000128
01_1250.csv contacted by ble
07_1251_th.csv contacted by ble
3 시각 : 2021-04-23 06:21:30.002000128
01_1250.csv contacted by ble
07_1251_th.csv contacted by ble
4 시각 : 2021-04-23 06:22:00.002000128
01_1250.csv contacted by ble
07_1251_th.csv contacted by ble
5 시각 : 2021-04-23 06:22:30.000999936
01_1250.csv contacted by ble
07_1251_th.csv contacted by ble
6 시각 : 2021-04-23 06:23:00.000999936
01_1250.csv contacted by ble
07_1251_th.csv contacted by ble
7 시각 : 2021-04-23 06:23:30.000999936
01_1250.csv contacted by ble
07_1251_th.csv contacted by ble
8 시각 : 2021-04-23 06:24:00.000999936
01_1250.csv contacted by ble
07_1251_th.csv contacted by ble
9 시각 : 2021-04-23 06:24:30.000999936
01_1250.csv contacted by ble
07_1251_th.csv contacted by ble
10 시각 : 2021-04-23 0